In [22]:
import os
import pickle

import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

from utils import load_embeddings, load_labels, compute_all_labels, aggregate_embeddings, aggregate_labels


# t-sne parameters
PARAMS_TSNE = {
    "yamnet":{ "peplexity": 30, "early_exaggeration": 5},
    "openl3":{ "peplexity": 3, "early_exaggeration": 9},
    "wav2vec":{ "peplexity": 80, "early_exaggeration": 12},
}

# Define colors
colors = {
    'Human': (229, 27, 32),
    'Human/Cough': (101, 19, 17),
    'Human/Movement': (248, 201, 193),
    'Human/Movement/Steps': (248, 201, 193),
    'Human/Music': (234, 105, 104),
    'Human/Voice': (178, 28, 28),
    'Industrial': (32, 75, 155),
    'Industrial/Construction': (134, 144, 200),
    'Industrial/Industry': (110, 109, 143),
    'Industrial/Vacuum': (204, 202, 216),
    'Industrial/Ventilation': (69, 74, 113),
    'Nature': (101, 179, 46),
    'Nature/Animals': (142, 194, 87),
    'Nature/Animals/Bird': (167, 206, 123),
    'Nature/Animals/Cat': (106, 138, 93),
    'Nature/Animals/Cricket': (66, 117, 63),
    'Nature/Animals/Dog': (162, 208, 162),
    'Nature/Animals/Sheep': (186, 191, 162),
    'Nature/Elements': (39, 53, 29),
    'Nature/Elements/Thunder': (110, 123, 102), 
    'Nature/Elements/Water': (97, 101, 66),
    'Nature/Elements/Wind': (42, 76, 38),
    'Nature/Vegetation': (72, 129, 49),
    'Signals': (238, 115, 22), 
    'Signals/Alarm': (242, 145, 73),
    'Signals/Bells': (174, 86, 22),
    'Signals/Klaxon': (251, 205, 169),
    'Signals/Siren': (206, 149, 103),
    'Things': (251, 224, 23),
    'Things/Ball': (253, 242, 184),
    'Things/Blind': (252, 243, 162),
    'Things/Door': (252, 248, 198),
    'Things/Movement': (252, 229, 91),
    'Things/Trolley': (251, 224, 50),
    'Transport': (45, 188, 238),
    'Transport/Motorized': (91, 153, 186),
    'Transport/Motorized/Air': (23, 180, 233),
    'Transport/Motorized/Rail': (139, 176, 203),
    'Transport/Motorized/Road': (23, 131, 170),
    'Transport/Non-motorized': (154, 211, 243)
}
colors = {k: tuple([c / 255 for c in v]) for k, v in colors.items()}


IMAGE_FOLDER = "figures/2d_plots"
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)

DATA_FOLDER: str = "data"
EMB_FOLDER: str = "data/embeddings"

loaded_embeddings = load_embeddings(EMB_FOLDER)
labels = load_labels(DATA_FOLDER)
all_labels = compute_all_labels(labels)

models = loaded_embeddings.keys()
datasets = labels.keys()

# let's modify embeddings to be {model: {dataset: np.array (total_frames, features)}}
# and labels to be {dataset: [list (tot n frames) of lists (labels in frame)]}
loaded_embeddings = aggregate_embeddings(loaded_embeddings)
labels = aggregate_labels(labels)


fig, axs = plt.subplots(3, 3, figsize=(18, 16), dpi=300)

for j, model in enumerate(models):
    image_folder = os.path.join(IMAGE_FOLDER, model)
    if not os.path.exists(image_folder):
        os.makedirs(image_folder)

    print(f"Plotting for model {model}")
    print(f"and saving them into {image_folder}")
    
    # we want a single array for every emb of the model (i.e., for each dataset)
    # and a list of list of labels, w same length as emb.shape[0].
    # Also, to do the plots, we need a dict {dataset: (start_idx, end_idx)}.
    # We will compute it while concatenating the embeddings
    idxs = {}
    start, end = 0, 0
    model_emb = []
    for dataset in datasets:
        e = loaded_embeddings[model][dataset]
        end += e.shape[0]
        idxs[dataset] = (start, end)
        start = end

        model_emb.append(e)
    model_emb = np.concatenate(model_emb)

    model_labels = []
    for dataset in datasets:
        model_labels.extend(labels[dataset])

    pca = PCA(n_components=50)
    pca_data = pca.fit_transform(model_emb)
    # compute tsne
    perpl = PARAMS_TSNE[model]["peplexity"]
    early_ex = PARAMS_TSNE[model]["early_exaggeration"]
    tsne = TSNE(n_components=2, perplexity=perpl, early_exaggeration=early_ex)
    tsne_data = tsne.fit_transform(pca_data)
    # we can plot
    pre_trained_model = model
    size = 3
    alpha = 0.3

    for i, dataset in enumerate(datasets):
        start, end = idxs[dataset]
        data_subset = tsne_data[start:end]
        lab_list_subset = model_labels[start:end]
        for lab in all_labels:
            lab_idx = [l_i for l_i, l in enumerate(lab_list_subset) if lab in l]
            if len(lab_idx) == 0:
                continue
            axs[j, i].scatter(
                x=data_subset[lab_idx, 0],
                y=data_subset[lab_idx, 1],
                label=lab,
                color=colors[lab],
                s=size,
                alpha=alpha,
            )
            
            #title1 = "Titolo linea 1"
            #title2 = "Titolo linea 2"

            #ax.text(.5, 1.05, title1, horizontalalignment='center', transform=ax.transAxes, fontsize=14, fontweight='bold')
            #ax.text(.5, 1.01, title2, horizontalalignment='center', transform=ax.transAxes, fontsize=12)

            #ax.plot([1, 2, 3, 4], [1, 4, 2, 3])
            #plt.show()

        #axs[j, i].set_title(dataset.replace("_", " ").title(), fontsize=18, pad=15)
        axs[j, i].text(.5, 1.02, dataset.replace("_", " ").title(), horizontalalignment='center', fontsize=18, transform=axs[j, i].transAxes)
        if i == 1:
            suptitle = "c) YAMNet" if pre_trained_model == "yamnet" else "a) OpenL3" if pre_trained_model == "openl3" else "b) Wav2Vec"
            axs[j, i].text(.5, 1.1, suptitle, horizontalalignment='center', fontsize=22, fontweight='bold', transform=axs[j, i].transAxes)

# The following two lines generate custom fake lines that will be used as legend entries:
colors_to_plot = {k: v for k, v in colors.items() if k in all_labels}
markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in colors_to_plot.values()]
fig.legend(markers, colors_to_plot.keys(), numpoints=1, loc="lower center", bbox_to_anchor=(0.5, 1.01), ncol=7, markerscale=1)

fig.subplots_adjust(hspace = 0.5)
plt.tight_layout()
model_folder = os.path.join(image_folder, 'tsne')
if not os.path.exists(model_folder):
    os.makedirs(model_folder)
plt.savefig("2demb.pdf", dpi=300, bbox_inches="tight")
plt.close()


Plotting for model openl3
and saving them into figures/2d_plots/openl3
Plotting for model wav2vec
and saving them into figures/2d_plots/wav2vec
Plotting for model yamnet
and saving them into figures/2d_plots/yamnet
